# Gene annotation
#### Description
This jupyter notebook contains a workflow to find the protein ID 'WP_011014180' of a specific gene (e.g. 'lysA')

Input: 
- **genes_to_find** <class 'list'> to search for their corresponding protein IDs.
- **model** <class 'cobra.core.model.Model'>

Output:
- **found_matches** <class 'dict'>


#### Usage:

```python
genes_to_find = ["ddh", "dapB", "lysA"]
found_matches = findGenes(genes_to_find, model)

found_matches

    {
        'ddh': [],
        'dapB': [{'id': 'WP_011014794', 'name': '4-hydroxy-tetrahydrodipicolinate reductase', 'similarity': None}],
        'lysA': [{'id': 'WP_011014180', 'name': 'diaminopimelate decarboxylase', 'similarity': 100.0}], 
    }

```


#### Steps of the algorithm

1. Building a local list **protein_id_to_gene_map** (.json file) containing all the published genes of **_C. glutamicum_ ATCC 13032** in RefSeq database
2. Searching through protein_id_to_gene_map for matches based on **gene name**
3. If genes still NOT found, collecting all the genes from homologous _C. glutamicum_ (10 strains available in KEGG)
4. Searching through them for the gene name, when found, quering its amino acid sequence
5. Searching through protein_id_to_gene_map for matches based on **AA sequence** homology

Author: 
`Haroun Bensaadi`

In [2]:
from colorama import init, Fore, Style
from collections import defaultdict
from Bio import Entrez, SeqIO, pairwise2
from IPython.display import Markdown
import requests
import time
import json
import re


def findGenes(genes_to_find, model):
    
    """
    MAIN METHOD USED TO FIND THE GENES AND THEIR CORRESPONDING PROTEIN ID
    
    - INPUT: genes_to_find (type: list) 
    - OUTPUT: found_matches (type: json)

    """
    global genes_not_found
    global found_homologous_genes
    
    found_genes_by_gene_name = []
    found_genes_by_aa_sequence = {}

    found_matches = {gene: [] for gene in genes_to_find}
    mapProteinIDtoGene()

    # 1. searching by gene_name
    for gene_name in genes_to_find:
        match_results = searchGene(gene=gene_name)    
        if match_results:
            found_genes_by_gene_name.append(gene_name)
            for match in match_results:
                found_matches[gene_name].append({"id": match[0], "name": match[1][1], "similarity": None})
                print(f"    ● {match[0]} [{match[1][1]}]")

    # 2. searching by AA sequence homolgy
    getDataBasedOnTaxonomy()
    genes_not_found = list(set(genes_to_find) - set(found_genes_by_gene_name))

    found_homologous_genes = getMatchedGenesByKEGGIdentifier()
    aa_sequence_results = fetchAAsequenceOfHomologousGenes(found_homologous_genes)

    print("\nSearching for AA sequence homology against genes from C. glutamicum ATCC 13032:")

    for gene_id, data in list(aa_sequence_results.items()): 
        for element in data:
            match_results = searchGene(seq= element["aa_seq"], similarity_threshold=0.50, kegg_id = element["kegg_id"])
            if match_results:
                for match in match_results:
                    found_matches[gene_id].append({"id": match[0], "name": match[2][1], "similarity": float(match[1].split("similarity: ")[1][:-1])})
                    print(f"    ● {match[0]} ({match[1]}) [{match[2][1]}]: {match[2][3][:35]}...")

    # Keeping only the match with highest similarity score:
    for gene, matches in found_matches.items():
        if len(matches) > 1:
            max_similarity = max(matches, key=lambda x: x['similarity'])
            found_matches[gene] = [max_similarity]

    return found_matches


# 1. Direct search through the collected list of annoctated genes:¶

def getFullGeneName(protein_id):
    for gene in model.genes:
        if protein_id in gene.id:
            return gene.id
        
def getProteinID(full_gene):
    return "WP_" + re.search(r"WP_(\d{9})", full_gene).group(1) if re.search(r"WP_(\d{9})", full_gene) else None

def fetchProteinID(protein_id, max_retries=50, delay_between_retries=1):
    for attempt in range(max_retries):
        try:
            Entrez.email = "hte.bensaadi@gmail.com"

            handle = Entrez.efetch(db="protein", id=protein_id, rettype="gb", retmode="text")
            record = SeqIO.read(handle, "genbank")
            handle.close()

            gene_name, product, organism, sequence = None, None, None, None

            for feature in record.features:
                if 'gene' in feature.qualifiers:
                    gene_name = feature.qualifiers['gene'][0]
                if 'product' in feature.qualifiers:
                    product = feature.qualifiers['product'][0]
                if 'organism' in feature.qualifiers:
                    organism = feature.qualifiers['organism'][0]

            sequence = str(record.seq)

            return protein_id, gene_name, product, organism, sequence

        except Entrez.HTTPError as e:
            if attempt < max_retries - 1:
                print(f"{protein_id}: retrying in {delay_between_retries} seconds...")
                time.sleep(delay_between_retries)
            else:
                print(f"Max retries reached. Unable to fetch data for {protein_id}")
                raise
        except Exception as e:
            print(f"Error: {e}")
            raise

def loadProteinIDToGeneMap():
    try:
        with open("Files/protein_id_to_gene_map.json", "r") as file:
            protein_id_to_gene_map = json.load(file)
    except FileNotFoundError:
        protein_id_to_gene_map = {}
    
    return protein_id_to_gene_map
    
def updateProteinIDToGeneMap(protein_id_to_gene_map):
    with open("Files/protein_id_to_gene_map.json", "w") as file:
        json.dump(protein_id_to_gene_map, file)
           
            
def getNumberOfUniqueGenes():
    
    unique_protein_ids = []
    
    for gene in model.genes:
        if getProteinID(gene.id) not in unique_protein_ids:
            unique_protein_ids.append(getProteinID(gene.id))
    return len(unique_protein_ids)
    
def mapProteinIDtoGene():
    global protein_id_to_gene_map
    
    print(f"Total number of unique genes in the model is: {getNumberOfUniqueGenes()} genes")
    
    protein_id_to_gene_map = loadProteinIDToGeneMap()

    if len(protein_id_to_gene_map) < 1:
        print(f"\nFetching RefSeq data for C. glutamicum ATCC 13032: ")        

        count_found_genes = 0
        for gene in model.genes:
            if "WP_" in gene.id:
                protein_id = getProteinID(gene.id)

                if protein_id not in protein_id_to_gene_map:
                    protein_id, gene_name, product, organism, sequence = fetchProteinID(protein_id)
                    protein_id_to_gene_map[protein_id] = [gene_name, product, organism, sequence]

                    updateProteinIDToGeneMap(protein_id_to_gene_map)

                    count_found_genes +=1

                    print(f"\n{protein_id}, {gene_name}, {product}, {organism}, {sequence}")

        print(f"Total number of fetched genes online is: {count_found_genes}")
    
    else:
        print(f"\nRefSeq data from  C. glutamicum ATCC 13032 is already stored locally: {len(protein_id_to_gene_map)+1} genes")        

# 2. Searching through AA sequence homology:

def calculate_similarity(seq1, seq2):
    alignments = pairwise2.align.globalxx(seq1, seq2, one_alignment_only=True)
    if alignments:
        alignment = alignments[0]
        alignment_length = max(len(alignment[0]), len(alignment[1]))
        similarity = alignment[2] / alignment_length
        return similarity
    return 0

def searchGene(gene=None, protein=None, seq=None, similarity_threshold=0.65, kegg_id=None):
    global protein_id_to_gene_map
    
    matches = []
    
    for protein_id, data in protein_id_to_gene_map.items():
        gene_name, product, organism, sequence = data
        
        if gene and gene_name and gene.lower() in gene_name.lower():
            matches.append((protein_id, data))
        
        if protein and product and protein.lower() in product.lower():
            matches.append((protein_id, data))
        
        if seq and sequence:
            similarity = calculate_similarity(seq.lower(), sequence.lower())            
            if similarity >= similarity_threshold:
                matches.append((protein_id, f"similarity: {round(similarity*100, 2)}%", data))
    
    if gene and len(matches)>0:
        print(f"\nMATCH: {gene} ->\n")
        return matches

    if seq and len(matches)>0:        
        print(f"\nMATCH: {getGeneNameFromKEGGID(kegg_id)} ({kegg_id}) ->\n")
        return matches

    if protein and len(matches)>0:
            print(f"\nMATCH: {protein} ->\n")
            return matches

    
def getDataFromProtreinID(searched_protein_id):
    protein_id_to_gene_map = loadMapFile()
    for protein_id, data in protein_id_to_gene_map.items():
        if searched_protein_id == protein_id:
            return data

def getGeneNameFromProteinID(searched_protein_id):
    for gene, reactions in list(gene_to_reactions.items()):
        if searched_protein_id == getProteinID(gene):
            return gene
    
def getReactionsFromProteinID(searched_protein_id):
    for gene, reactions in list(gene_to_reactions.items()):
        if searched_protein_id == getProteinID(gene):
            return reactions

        
def loadDataFromTaxonomySearch():
    try:
        with open("Files/data_from_taxonomy_search.json", "r") as file:
            data_from_taxonomy_search = json.load(file)
    except FileNotFoundError:
        data_from_taxonomy_search = []
    return data_from_taxonomy_search
    
def saveDataFromTaxonomySearch(data_from_taxonomy_search):
    with open("Files/data_from_taxonomy_search.json", "w") as file:
        json.dump(data_from_taxonomy_search, file)
        
def getDataBasedOnTaxonomy():
    global data_from_taxonomy_search
    taxonomy_codes = {
        "cgl": "Corynebacterium glutamicum ATCC 13032 (Kyowa Hakko)",
        "cgb": "Corynebacterium glutamicum ATCC 13032 (Bielefeld)",
        "cgu": "Corynebacterium glutamicum K051",
        "cgt": "Corynebacterium glutamicum R",
        "cgs": "Corynebacterium glutamicum SCgG1",
        "cgg": "Corynebacterium glutamicum SCgG2",
        "cgm": "Corynebacterium glutamicum MB001",
        "cgj": "Corynebacterium glutamicum ATCC 21831",
        "cgq": "Corynebacterium glutamicum AR1",
        "cgx": "Corynebacterium glutamicum B253"}

    data_from_taxonomy_search = loadDataFromTaxonomySearch()
    
    if len(data_from_taxonomy_search) < 1:
        print(f"Fetching KEGG data for {len(taxonomy_codes)} C. glutamicum strain: \n")
        
        for taxonomy_code in taxonomy_codes:
            search_url = f"http://rest.kegg.jp/find/genes/{taxonomy_code}"

            try:
                print(f"Getting data for {taxonomy_codes[taxonomy_code]}")
                response = requests.get(search_url)
                if response.status_code == 200:
                    data_from_taxonomy_search.append(response.text.split('\n'))
                else:
                    print(f"Error: {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"Request Exception: {e}")

        saveDataFromTaxonomySearch(data_from_taxonomy_search)
    else:
        print(f"\nKEGG data from other C. glutamicum strains is already stored locally: {len(data_from_taxonomy_search)} strains\n")
            
def getAminoAcidsSequence(gene_id):
    response = requests.get(f"http://rest.kegg.jp/get/{gene_id}")
    
    if response.status_code == 200:        
        lines = response.text.split('\n')
        amino_acid_sequence = ""
        is_aa_sequence_section = False

        for line in lines:
            if line.startswith("AASEQ"):
                is_aa_sequence_section = True
            elif line.startswith("NTSEQ"):
                is_aa_sequence_section = False
            elif is_aa_sequence_section and line.strip(): 
                amino_acid_sequence += line.strip()
        return amino_acid_sequence

    else:
        return f"Error: {response.status_code}"
    
    
def getMatchedGenesByKEGGIdentifier():
    global data_from_taxonomy_search
    global genes_not_found
    
    found_homologous_genes = {}
    
    for gene_name_to_search in genes_not_found:
        for data in data_from_taxonomy_search:
            for entry in data:
                
                if re.compile(fr'{gene_name_to_search.lower()}.*?;').search(entry.lower()):
                    if gene_name_to_search not in found_homologous_genes:
                        found_homologous_genes[gene_name_to_search] = [entry]
                    else:
                        found_homologous_genes[gene_name_to_search].append(entry)

        try:
            print(f"    Homologous genes for {gene_name_to_search}: {len(found_homologous_genes[gene_name_to_search])} results")
        except:
            pass
            
    return found_homologous_genes


def fetchAAsequenceOfHomologousGenes(found_homologous_genes):
    
    aa_sequence_results = {}
    
    print("\nFetching AA sequence for the homologous genes: \n")

    for gene_name, entries in list(found_homologous_genes.items()):
        for entry in entries:
            gene_id_kegg = entry.split("; ")[0].split("\t")[0]
            gene_name_kegg = entry.split("; ")[0].split("\t")[1]
            try:
                gene_description_kegg = entry.split("; ")[1]
            except:
                pass

            aa_seq = getAminoAcidsSequence(gene_id_kegg)

            print(f"    {gene_name_kegg} ({gene_id_kegg}): {aa_seq[:25]}...")

            if gene_name not in aa_sequence_results:
                aa_sequence_results[gene_name] = [{"kegg_id": gene_id_kegg, "aa_seq": aa_seq}]
            else:
                aa_sequence_results[gene_name].append({"kegg_id": gene_id_kegg, "aa_seq": aa_seq})

    return aa_sequence_results

def getGeneNameFromKEGGID(kegg_id_to_find):
    for gene_name, entries in list(found_homologous_genes.items()):
        for entry in entries:
            kegg_id = entry.split("; ")[0].split("\t")[0]
            gene_name = entry.split("; ")[0].split("\t")[1]
            if kegg_id_to_find == kegg_id:
                return gene_name


def printReport(found_matches):    
    GENES = {}
    
    print("\n\n\nSummary:\n")
    
    for gene, matches in found_matches.items():
        if len(matches) > 0 and matches[0]['similarity'] == None:
            GENES[gene] = getFullGeneName(matches[0]['id'])
            print(f"    ● {gene}: {matches[0]['id']}")
    
    for gene, matches in found_matches.items():    
        if len(matches) > 0 and matches[0]['similarity']:
            GENES[gene] = getFullGeneName(matches[0]['id'])
            print(f"    ● {gene}: {matches[0]['id']} (similarity {matches[0]['similarity']}%)")
    
    for gene, matches in found_matches.items():
        if len(matches) == 0:
            print("\nMissing genes:\n")    
            print(f"    ● {gene}: None")

    return GENES

def printText(text, color=Fore.WHITE, style=Style.BRIGHT):
    print(style + color + text + Style.RESET_ALL)
    
def printReaction(reaction):
    substrates = ' + '.join([f"{metabolite.name}" for metabolite in reaction.reactants])
    products = ' + '.join([f"{metabolite.name}" for metabolite in reaction.products])
    reaction_string = f"{substrates}  -->  {products}"
    return reaction_string